Suzan Verberne 17.10.2023 1
Assignment 2: sequence labelling
Text mining course
This is a hand-in assignment for groups of two students. Send in via Brightspace before or on
Tuesday November 7:
• Submit your report as PDF and your python code as separate file. Don’t upload a zip file
containing the PDF (the Python code might be zipped if it consists of multiple files).
• Your report should not be longer than 3 pages.
• Do not copy text from sources (other groups, web pages, generative models such as
chatGPT). Turnitin is enabled and a large overlap will be reported to the Board of Examiners.
Goals of this assignment
• You can pre-process existing annotated text data into the data structure that you need for
classifier learning.
• You can perform hyperparameter optimization.
• You can perform a sequence labelling task with annotated data in Huggingface.
• You can evaluate sequence labelling with the suitable evaluation metrics.
Preliminaries
• You have followed the Huggingface tutorial on token classification
https://huggingface.co/learn/nlp-course/chapter7/2 and its preliminaries (exercises week 6
and 7).
• You have all the required Python packages installed and Python 3.10.
We are going to train an NER classifier for the task “Emerging and Rare entity recognition” from the
Workshop on Noisy User-generated Text (W-NUT). The description of the task can be found at
https://noisy-text.github.io/2017/emerging-rare-entities.html (I put the data itself on Brightspace)
Tasks
1. Download W-NUT_data.zip from the Brightspace assignment and unzip the directory. It
contains 3 IOB files: wnut17train.conll (train), emerging.dev.conll (dev),
emerging.test.annotated (test).
2. Convert the IOB data to the correct data structure for token classification in Huggingface
(words and labels like the conll2023 data in the tutorial) and align the labels with the tokens.
Note that since you are working with a custom dataset, the data conversion is a necessary
step for using the Huggingface training function.
Suzan Verberne 17.10.2023 2
3. Set up the evaluation correctly for the W-NUT test set, following the tutorial.
4. Fine-tune a model with the default hyperparameter settings on the train set and evaluate
the model on the test set. These are your baseline results.
5. Set up hyperparameter optimization with the AdamW optimizer. During optimization, use
the dev set as validation. After the model has been optimized, evaluate the result on the test
set.
6. Extend the evaluation function so that it shows the Precision, Recall and F-score for each of
the entity types (person, location, etc.) on the test set. Include the metrics for the B-label of
the entity type, the I-label, and the full entities. Look up the definitions of macro- and micro-
average scores and compute the macro- and micro average F1 scores over all entities.
Write a report of at most 3 pages in which you:
• describe the task and the data (give a few statistics. What are the entity types?) and briefly
describe two challenges of the task and the data.
• show your results:
o a results table with both the baseline results and the results after hyperparameter
optimization (do not report results on the dev set, only on the test set): a table with
Precision, Recall, F-score for both settings.
o a table with the results after hyperparameter optimization for the different entity
types (Precision, Recall, F-score for B, I, and the full entities), and the macro- and
micro F1 scores.
• write brief conclusions. Address the following questions:
o What is the effect of hyperparameter optimization on the quality of the model?
o What does the difference between scores for different entity types tell you?
o Where does the difference between macro- and micro-averaged F1 scores come
from?
Grading
Maximum 2 points for each of the following criteria:
• General: length correct (2-3 pages) and proper writing + formatting
• Description of the task and the data, with description of 2 challenges
• Baseline results with default hyperparameter settings and results with optimized
hyperparameter settings
• Results after hyperparameter optimization for the different entity types
• Sensible conclusions, briefly addressing the questions listed above.

In [4]:
import datasets

In [5]:
names=[
    "O",
    "B-corporation",
    "I-corporation",
    "B-creative-work",
    "I-creative-work",
    "B-group",
    "I-group",
    "B-location",
    "I-location",
    "B-person",
    "I-person",
    "B-product",
    "I-product"
    ]

In [6]:
train, valid, test = 'wnut17train.conll', 'emerging.dev.conll', 'emerging.test.annotated'
path = 'W-NUT_data'

raw_datasets = {}
for file, name in zip([train, valid, test], ['train', 'validation', 'test']):
    id = 0
    raw_datasets[name] = {'id': [], 'tokens': [], 'ner_tags': []}
    with open(f'{path}/{file}', 'r') as f:
        tokens, ner_tags = [], []
        for line in f:
            try:
                token, ner_tag = line.split()
                tokens.append(token)
                ner_tags.append(names.index(ner_tag))
            except:
                raw_datasets[name]['id'].append([id for _ in range(len(tokens))])
                raw_datasets[name]['tokens'].append(tokens)
                raw_datasets[name]['ner_tags'].append(ner_tags)
                id += 1
                tokens, ner_tags = [], []
    raw_datasets[name] = datasets.Dataset.from_dict(raw_datasets[name])

raw_datasets = datasets.DatasetDict(raw_datasets)
display(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

# Processing the data

In [9]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
tokenizer.is_fast

True

In [11]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)

In [12]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [13]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [15]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map: 100%|██████████| 1287/1287 [00:00<00:00, 7798.05 examples/s]


# Fine-tuning the model with keras

## Data collation

In [16]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [18]:
!pip install evaluate

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

## Defining the model

In [ ]:
id2label = {i: label for i, label in enumerate(names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

##  Fine-tuning the model

In [ ]:
from huggingface_hub import interpreter_login

interpreter_login()

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

## Fit model

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-finetuned-ner", tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=3,
)

### Evaluate w/o hyperparameter tuning

In [ ]:
import evaluate
import numpy as np

all_predictions = []
all_labels = []
for batch in tf_test_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(names[predicted_idx])
            all_labels.append(names[label_idx])

In [ ]:
def TFcompute_metrics(preds, labels):

    metric = evaluate.load("seqeval")
    all_metrics = metric.compute(predictions=[preds], references=[labels])
    print("\\begin table}[h]")
    print("\\centering")
    print("\\begin{tabular}{lrrrr}")
    print("\\toprule")
    print(" & \\textbf{Precision} & \\textbf{Recall} & \\textbf{F1} & \\textbf{Number} \\\\")
    print("\\midrule")
    for key, value in all_metrics.items():
        if "overall" not in key:
            print(f"{key.title()} & {value['precision']:.2f} & {value['recall']:.2f} & {value['f1']:.2f} & {value['number']} \\\\")
    print("\\midrule")
    print(" & \\textbf{Overall precision} & \\textbf{Overall recall} & \\textbf{Overall F1} & \\textbf{Overall accuracy} \\\\")
    print("\\midrule")
    print(f" & {all_metrics['overall_precision']:.2f} & {all_metrics['overall_recall']:.2f} & {all_metrics['overall_f1']:.2f} & {all_metrics['overall_accuracy']:.2f} \\\\")
    print("\\bottomrule")
    print("\\end{tabular}")
    print("\\caption{Results of the NER model on the test set.}")
    print("\\label{tab:ner-results}")
    print("\\end{table}")
    
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

TFcompute_metrics(all_predictions, all_labels);

## Tuning the NER model

In [ ]:
import keras_tuner as kt

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

tuner = kt.Hyperband(model,
                     objective='val_loss',
                     max_epochs=10,
                     factor=3,
                     directory='tuned_model',
                     project_name='hp_search_ner')